In [20]:
import torch

model = torch.load('./mlm_finetune_output/model/epoch_0/pytorch_model.bin', map_location='cpu')
model.keys()

dict_keys(['bert.embeddings.position_ids', 'bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.0.output.LayerNorm.weight

In [21]:
import re
def clean_text(line):
    line = re.sub(r'-+',' ',line)
    #line = re.sub(r'[^a-zA-Z, ]+'," ",line)
    line = re.sub(r'[ ]+'," ",line)
    line += "."
    return line

In [22]:

text = 'In contrast, further deletion of the C-terminal transactivation domain in the Pax5 mutants B8 and B9 can abolish transcriptional stimulation, whereas internal deletion of the conserved octapeptide motif (OP) or the partial homeodomain (HD) of Pax5 did not have any effect (Figure 3B).'
print(clean_text(text))

In contrast, further deletion of the C terminal transactivation domain in the Pax5 mutants B8 and B9 can abolish transcriptional stimulation, whereas internal deletion of the conserved octapeptide motif (OP) or the partial homeodomain (HD) of Pax5 did not have any effect (Figure 3B)..


In [23]:
words = re.split(r'([.,;\s])', text)    
words = [word for word in words if word != ' ' and word != ''] 

print(words)

['In', 'contrast', ',', 'further', 'deletion', 'of', 'the', 'C-terminal', 'transactivation', 'domain', 'in', 'the', 'Pax5', 'mutants', 'B8', 'and', 'B9', 'can', 'abolish', 'transcriptional', 'stimulation', ',', 'whereas', 'internal', 'deletion', 'of', 'the', 'conserved', 'octapeptide', 'motif', '(OP)', 'or', 'the', 'partial', 'homeodomain', '(HD)', 'of', 'Pax5', 'did', 'not', 'have', 'any', 'effect', '(Figure', '3B)', '.']


In [24]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = BertForMaskedLM.from_pretrained("dmis-lab/biobert-v1.1")

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
print("inputs", inputs)
print("len(inputs.input_ids[0])", len(inputs.input_ids[0]))
with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# mask labels of non-[MASK] tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)
print("labels", labels)
print("len(labels[0])", len(labels[0]))
outputs = model(**inputs, labels=labels)
round(outputs.loss.item(), 2)

/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-05 01:37:10.021526: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 01:37:10.079390: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 01:37:10.905702: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: 

inputs {'input_ids': tensor([[ 101, 1109, 2364, 1104, 1699, 1110,  103,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
len(inputs.input_ids[0]) 9
labels tensor([[-100, -100, -100, -100, -100, -100, 2123, -100, -100]])
len(labels[0]) 9


11.3

In [25]:

import numpy as np
import random
def get_random_indices(sentence):
    # Get the length of the sentence
    sentence_length = len(sentence.split())
  
    masked_idx = random.sample(range(sentence_length), 10)
    print("masked_idx", masked_idx)
    # Shuffle the indices of the sentence
    indices = np.arange(0, sentence_length-1, 1)
    np.random.shuffle(indices)

    # Select the first 10 unique indices
    random_indices = indices[:10]

    # Ensure that the selected indices are distinct
    while len(np.unique(random_indices)) < 10:
        np.random.shuffle(indices)
        random_indices = indices[:10]

    return random_indices


sentences = [
    "This is a sample sentence This is a sample sentence This is a sample sentence.",
    "Another example sentence with more words This is a sample sentence This is a sample sentence.",
    "A third sentence to demonstrate the process This is a sample sentence This is a sample sentence.",
]

for sentence in sentences:
    random_indices = get_random_indices(sentence)
    print(f"Sentence: {sentence}")
    print(f"Random Indices: {random_indices}")


masked_idx [11, 14, 0, 1, 6, 4, 5, 10, 3, 7]
Sentence: This is a sample sentence This is a sample sentence This is a sample sentence.
Random Indices: [ 1  4  3  2  6  9 12  7 11  0]
masked_idx [4, 11, 12, 3, 0, 6, 10, 8, 15, 7]
Sentence: Another example sentence with more words This is a sample sentence This is a sample sentence.
Random Indices: [12 10  3 14  6  5  1  8 13  4]
masked_idx [0, 14, 8, 16, 7, 1, 13, 4, 12, 5]
Sentence: A third sentence to demonstrate the process This is a sample sentence This is a sample sentence.
Random Indices: [ 9  5 13  4  2  3  1 10 14  0]


In [23]:
MAX_SEQ_LEN = 70

def is_in_vocab(token, tokenizer):
    '''
    Function to check if a token is in the vocabulary
    '''
    return 1 if token in tokenizer.vocab.keys() else 0
import copy
def masking_sentence_word(words, tokenizer):
    '''
    Function to mask random token in a sentence and return the masked sentence and the corresponding label ids
    '''
    except_tokens = [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]
    
    masked_idx = random.sample(range(len(words)), 15)
    print("masked_idx", masked_idx)
    
    # create 10 sentences with 10 masked tokens
    
    sen_10 = []
    lable_10 = [-100] * MAX_SEQ_LEN 
  
    for i in range(15): 
        tmp_sen = copy.deepcopy(words)
        tmp_label = copy.deepcopy(labels)
        if len(sen_10) < 10:
            masked_token = tmp_sen[masked_idx[i]]
            if (masked_token not in except_tokens) and is_in_vocab(tmp_sen[masked_idx[i]], tokenizer):
                print(masked_idx[i])
                tmp_label[masked_idx[i]] = tokenizer.convert_tokens_to_ids(tmp_sen[masked_idx[i]])
                tmp_sen[masked_idx[i]] = tokenizer.mask_token
            
            sen_10.append(tmp_sen)
            lable_10.append(tmp_label)
        else :
            break
        # t chạy cell éo dc
            
    return sen_10, lable_10

In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

sentences = [["This is the first sentence."], ['This', 'is' ,the sencond sentence.']] 
encoded_inputs = tokenizer(sentences, padding="max_length", truncation=True)

print(encoded_inputs)


{'input_ids': [[101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [3]:
labels = [-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,1719,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100]

# Find the index of the first element not equal to -100
index = next((i for i, x in enumerate(labels) if x != -100), None)

# Print the index
print(index)

21


In [2]:
import pandas as pd
labels = [-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,1719,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100]

# Create a pandas dataframe
df = pd.DataFrame({"values": labels})

# Find the index of the first element not equal to -100
index = df.loc[df["values"] != -100].index[0]

# Print the index
print(index)

21


In [3]:
import torch
import torch.nn.functional as F

# Example shapes
input_ids = torch.randn(32, 85)  # Assuming 32 batches, each with a sequence length of 85
logits = outputs.logits[0]  # Example logits, shape [85, 28996]
labels = torch.randint(0, 28996, (32, 85))  # Assuming labels for each position in the sequence

# Ensure that logits match the number of classes in your task
num_classes = logits.size(1)

# Transpose logits to be [sequence_length, batch_size, num_classes]
logits = logits.transpose(0, 1)

# Flatten the logits and labels for the loss calculation
logits_flat = logits.contiguous().view(-1, num_classes)
labels_flat = labels.view(-1)

# Assuming a simple sequence classification task
# You can use CrossEntropyLoss along the sequence dimension
loss_fn = torch.nn.CrossEntropyLoss()

# Calculate the loss
loss = loss_fn(logits_flat, labels_flat)

# Your training/update step here


NameError: name 'outputs' is not defined

In [7]:
import torch

# Example tensor with size [32, 85]
original_tensor = torch.randn(32, 85)
print(original_tensor)  
# Sum along the second dimension (axis 1)
summed_tensor = torch.sum(original_tensor, dim=1)
print(summed_tensor)
# Check the size of the resulting tensor
print(summed_tensor.size())  # Should print torch.Size([32])



tensor([[-0.2069, -0.0487, -0.9721,  ..., -0.0636, -0.5318, -0.4599],
        [ 0.2312,  0.3660, -1.0198,  ...,  0.9095,  0.4571, -0.6176],
        [-0.2711,  1.2842, -0.5105,  ...,  0.2480,  0.2303,  1.1980],
        ...,
        [-0.2934, -0.6806,  0.2976,  ...,  1.2182,  0.1279,  1.4468],
        [-0.0404,  0.2452,  1.8691,  ...,  0.3390, -0.4477, -0.5021],
        [ 0.6592, -0.6855, -2.5391,  ..., -0.2193,  0.9167,  0.8985]])
tensor([-5.1780,  8.0060, -4.2243, -4.0286, -2.2999, -3.1989, -0.1657,  2.9415,
         3.4626,  4.1581, -3.5055, -0.0787, -1.9848, -8.1679, 11.1749,  1.9731,
        -9.6414, -6.0795,  3.4942, 16.2574, 19.9127, -6.1354,  2.3748,  1.3002,
        -7.9017,  6.0350, -2.2623, -0.6856, 11.2451, -6.6025,  6.6847, -8.9130])
torch.Size([32])


In [6]:
import torch
# create a tensor of size ([32, 85, 28996]) and convert it with size ([32, 28996])

# another way to do it without sum
tensor = torch.randn(32, 85, 28996)
print(tensor)
print(tensor.size())
#tensor = tensor[:, 0, :]
tensor = tensor.view(-1, 28996)
print(tensor)
print(tensor.size())


tensor([[[ 4.9110e-01,  9.0305e-01, -1.4568e+00,  ...,  1.8432e+00,
          -4.2961e-01,  1.2584e+00],
         [ 8.2205e-01,  4.1328e-01,  1.6755e+00,  ...,  4.8675e-01,
          -9.0687e-01,  1.0005e+00],
         [ 8.9015e-04, -1.6477e-01,  2.3290e+00,  ...,  1.8179e-01,
           9.0725e-01,  6.4934e-01],
         ...,
         [ 5.1636e-01,  2.3479e-01, -8.3017e-01,  ...,  1.5314e-01,
          -2.3371e-01,  6.0339e-01],
         [-7.8925e-01, -5.6791e-01,  1.2848e-01,  ..., -1.2228e+00,
           8.3731e-01,  2.2944e-01],
         [ 1.8019e+00, -9.2217e-01, -2.0894e-01,  ...,  8.1735e-01,
           4.0779e-01,  9.6223e-01]],

        [[ 5.6251e-01,  5.0760e-01,  4.0024e-01,  ..., -5.7665e-01,
           3.2209e-01,  1.8760e-01],
         [ 3.3259e-01, -1.8239e-01, -1.3535e+00,  ..., -1.2392e+00,
          -1.1423e+00,  5.8895e-01],
         [ 2.9254e-01, -1.6263e+00,  5.8284e-01,  ..., -1.3539e-01,
          -3.8462e-01,  1.7225e+00],
         ...,
         [ 2.1157e+00,  3